In [4]:
def scoring(csvfile):
    import psutil
    import os
    import time
    
    #System status; 
    print('Initial memory:')
    print(psutil.virtual_memory().percent/100)
    start_time = time.time()
    
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    import seaborn as sns
    %matplotlib inline
    import re
    import string
    import pickle

    #from sklearn.cross_validation import train_test_split
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn import linear_model
    from sklearn import preprocessing
    from sklearn.metrics import mean_squared_error, r2_score
    
    #Pre-processing; 
    data=pd.read_csv(csvfile)
    
    constant_feat=['x067', 'x094', 'x095', 'x096'] #assume holdout data also has these 4 features w/ only constant values.
    data.drop(constant_feat, axis=1, inplace=True)
    
    with open('trainmedian.pickle', 'rb') as f:
        data.fillna(pickle.load(f), inplace=True)

    #Predictions; 
    X=data.drop('y', axis=1)
    y=data['y']  
    
    with open('wh_pred.pickle', 'rb') as f:
        regr=pickle.load(f)
        y_pred = regr.predict(X)
    
    #Evaluations; 
    y_preddummy=(abs(y_pred-y)<=3)
    RSME=np.sqrt(mean_squared_error(y, y_pred))
    R2=r2_score(y, y_pred)
    Accu=y_preddummy.value_counts()/len(y_preddummy)
    perf = [RSME, R2, Accu[True]]
    performance_metric = pd.DataFrame(perf, index=['RSME', 'R-squared', 'Accuracy'], columns=['Value']) #smaller, bigger, bigger; 
    
    #Outputs; 
    pd.DataFrame(performance_metric).to_csv('performance_metric.txt', sep='\t', header=True, index=True)
    pd.DataFrame(y_pred).to_csv('y_pred.txt', sep='\t', header=False, index=True)
    
    #System status; 
    print('Memory after model output:')
    print(psutil.virtual_memory().percent/100)
    print("--- Processing time %s seconds ---" % (time.time() - start_time))

In [5]:
scoring('holdout.csv') 

Initial memory:
0.634
Memory after model output:
0.693
--- Processing time 5.268308877944946 seconds ---


In [3]:
'''
Summary of the analysis
1. Assumptions 
    Both the regressors and the imputed median values of the training dataset should be applied to future hold-out data together. 
    Also assume the hold-out dataset has the same data problems with the data provided: 4 single valued features ('x067', 'x094', 'x095', 'x096').

2. Steps and algorithms
    First remove 4 single-valued features, which provide no prediction power to the model: 'x067', 'x094', 'x095', 'x096'.
    Then split the dataset into 70% trainig and 30% test.
    For features with missing values, impute the median values of the corresponding feature from available training dataset into both training and test dataset to make the dataset fully available to use.
    Due to the limited computing power of my personal computer, here adopted only the model of RandomForestRegressor with 500 trees. 
    Three performance metrics are adopted: RSME, R-squared, accuracy rate (considering absolute error within 3 as 'accurate').
    Save the median values and regressor in pickle files, to use to run in a separate file for future scoring. 

3. My personal computer hardware specification:
    RAM: 4 GB 1600 MHz DDR3
    Processor: 1.4 GHz Intel Core i5

4. Estimated memory time needed by the program 
    Running 100,000 data points as test, 15% of virtual memory and time of 62 seconds have been used on the computer (specified above).

5. Output files generated by the program
    Two text files are created: 
        y_pred.txt will include 100,000 predictions of the hold-out dataset.
        performance_metric.txt will include 3 performance measures. 

6. To run this program: 
    Save 3 files at the same location with hold-out csv file: wh_pred.pickle, trainmedian.pickle, Scoring_from_scratch.py. 
    Providing the name of hold-out csv file to the 'scoring' function of the program (Scoring_from_scratch.py), the program will run and generate 2 output files. 
'''


"\nSummary of the analysis\n1. Assumptions \n    Both the regressors and the imputed median values of the training dataset should be applied to future hold-out data together. \n    Also assume the hold-out dataset has the same data problems with the data provided: 4 single valued features ('x067', 'x094', 'x095', 'x096').\n\n2. Steps and algorithms\n    First remove 4 single-valued features, which provide no prediction power to the model: 'x067', 'x094', 'x095', 'x096'.\n    Then split the dataset into 70% trainig and 30% test.\n    For features with missing values, impute the median values of the corresponding feature from available training dataset into both training and test dataset to make the dataset fully available to use.\n    Due to the limited computing power of my personal computer, here adopted only the model of RandomForestRegressor with 500 trees. \n    Three performance metrics are adopted: RSME, R-squared, accuracy rate (considering absolute error within 3 as 'accurate')